In [ ]:
!pip install mediapipe opencv-python-headless

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mediapipe as mp
from google.colab import files
import os
from datetime import datetime
import json
from PIL import Image
import math


In [ ]:
# ============================================================================
# Conexión con Google Drive y preparación de carpetas del proyecto
# ============================================================================

import os

try:
    # Conectar Google Colab con tu Google Drive
    from google.colab import drive
    drive.mount('/content/drive')  # Se abrirá una ventana para autorizar el acceso

    # Ruta principal del proyecto en Drive
    proyecto_path = "/content/drive/MyDrive/Analisis_Osteoforense"

    # Crear carpetas del proyecto si no existen
    os.makedirs(os.path.join(proyecto_path, "imagenes"), exist_ok=True)
    os.makedirs(os.path.join(proyecto_path, "resultados"), exist_ok=True)

    # Mensajes para confirmar
    print("Google Drive se ha montado correctamente.")
    print(f"Carpeta principal del proyecto: {proyecto_path}")

except Exception as e:
    # Si algo falla, trabajamos en almacenamiento local
    print("No se pudo montar Google Drive. Se usará almacenamiento local.")
    print(f"Detalles del error: {e}")

    # Ruta alternativa en el entorno local de Colab
    proyecto_path = "/content/Analisis_Osteoforense"

    # Crear carpetas locales si no existen
    os.makedirs(os.path.join(proyecto_path, "imagenes"), exist_ok=True)
    os.makedirs(os.path.join(proyecto_path, "resultados"), exist_ok=True)

    print(f"Carpeta local creada: {proyecto_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive se ha montado correctamente.
Carpeta principal del proyecto: /content/drive/MyDrive/Analisis_Osteoforense


In [ ]:
# ============================================================================
# Configuración de MediaPipe Face Mesh para análisis osteoforense
# ============================================================================

import mediapipe as mp  # Asegúrate de haberlo importado antes

# --- 1️Configurar el modelo Face Mesh ---
# static_image_mode=True  → Procesa imágenes estáticas (no video)
# max_num_faces=1         → Solo detecta un rostro
# refine_landmarks=True   → Más precisión en ojos, labios y cejas
# min_detection_confidence=0.5 → Confianza mínima para detección
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5
)

# --- 2️Mapeo de landmarks osteoforenses ---
# Diccionario que relaciona los puntos anatómicos clave con sus índices en MediaPipe
LANDMARK_MAPPING = {
    'glabella': 9,          # Punto más prominente de la frente
    'nasion': 6,            # Raíz nasal entre cejas
    'pronasale': 2,         # Punta de la nariz
    'subnasale': 2,         # Base de la nariz
    'pogonion': 175,        # Punto más anterior del mentón
    'gnathion': 18,         # Punto más inferior de la barbilla
    'endocanthion_L': 133,  # Canto interno del ojo izquierdo
    'endocanthion_R': 362,  # Canto interno del ojo derecho
    'ectocanthion_L': 33,   # Canto externo del ojo izquierdo
    'ectocanthion_R': 263,  # Canto externo del ojo derecho
    'alare_L': 129,         # Ala nasal izquierda
    'alare_R': 358,         # Ala nasal derecha
    'cheilion_L': 61,       # Comisura labial izquierda
    'cheilion_R': 291,      # Comisura labial derecha
    'labiale_superius': 13, # Punto medio del labio superior
    'labiale_inferius': 14, # Punto medio del labio inferior
    'stomion': 12,          # Contacto entre los labios
    'zygion_L': 116,        # Punto lateral del pómulo izquierdo
    'zygion_R': 345,        # Punto lateral del pómulo derecho
    'gonion_L': 172,        # Ángulo mandibular izquierdo
    'gonion_R': 397         # Ángulo mandibular derecho
}

# --- 3️Definición de medidas osteoforenses ---
# Cada entrada: abreviatura : (punto1, punto2, descripción)
MEDIDAS_OSTEOFORENSES = {
    'g_n': ('glabella', 'nasion', 'Altura glabelar'),
    'n_prn': ('nasion', 'pronasale', 'Longitud dorso nasal'),
    'n_sn': ('nasion', 'subnasale', 'Altura nasal total'),
    'sn_pg': ('subnasale', 'pogonion', 'Altura tercio inferior'),
    'g_gn': ('glabella', 'gnathion', 'Altura facial total'),
    'al_al': ('alare_L', 'alare_R', 'Ancho nasal'),
    'en_en': ('endocanthion_L', 'endocanthion_R', 'Distancia intercanthal'),
    'ex_ex': ('ectocanthion_L', 'ectocanthion_R', 'Distancia bi-ectocanthal'),
    'ch_ch': ('cheilion_L', 'cheilion_R', 'Ancho de boca'),
    'zy_zy': ('zygion_L', 'zygion_R', 'Ancho bizigomático'),
    'go_go': ('gonion_L', 'gonion_R', 'Ancho bigonial'),
    'ojo_izquierdo': ('endocanthion_L', 'ectocanthion_L', 'Ancho ojo izquierdo'),
    'ojo_derecho': ('endocanthion_R', 'ectocanthion_R', 'Ancho ojo derecho'),
    'ls_li': ('labiale_superius', 'labiale_inferius', 'Altura labial'),
    'sn_sto': ('subnasale', 'stomion', 'Altura labio superior')
}

# --- 4️Mensajes informativos ---
print("Configuración de MediaPipe completada.")
print(f"Landmarks definidos: {len(LANDMARK_MAPPING)}")
print(f"Medidas osteoforenses a calcular: {len(MEDIDAS_OSTEOFORENSES)}")


Configuración de MediaPipe completada.
Landmarks definidos: 21
Medidas osteoforenses a calcular: 15


In [ ]:
# ============================================================================
# FUNCIÓN: CARGAR IMAGEN PARA ANÁLISIS
# ============================================================================
from google.colab import files
import numpy as np
import cv2

def cargar_imagen():
    """
    Permite subir una imagen desde tu dispositivo y la prepara
    para su análisis con OpenCV.

    Retorna:
        imagen (ndarray): Imagen en formato OpenCV (BGR)
        filename (str): Nombre del archivo cargado
    """
    # --- 1️Solicitar al usuario subir una imagen ---
    print("PASO 1: Cargar imagen para análisis")
    print("Selecciona la imagen que deseas analizar:")

    uploaded = files.upload()  # Abre selector de archivos en Colab

    if not uploaded:
        print("No se subió ninguna imagen.")
        return None, None

    # --- 2️Obtener datos del archivo cargado ---
    filename = list(uploaded.keys())[0]           # Nombre del archivo
    image_data = uploaded[filename]               # Datos binarios

    # --- 3️Convertir a formato OpenCV ---
    nparr = np.frombuffer(image_data, np.uint8)
    imagen = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    if imagen is None:
        print("Error al decodificar la imagen.")
        return None, None

    # --- 4️Mostrar información de la imagen ---
    alto, ancho, canales = imagen.shape
    print("Imagen cargada exitosamente.")
    print(f"   📏 Dimensiones: {ancho} x {alto} píxeles")
    print(f"Archivo: {filename}")

    # --- 5️⃣ Guardar imagen en carpeta del proyecto ---
    # (Se usa un hash o nombre fijo, puedes personalizarlo)
    hash_imagen = "0aad2507592c642223603f2e46cf1bdc601f8136fd5e7a4cc4cec59f844a68b5"
    ruta_guardado = f"{proyecto_path}/imagenes/{hash_imagen}.jpg"
    cv2.imwrite(ruta_guardado, imagen)
    print(f"Imagen guardada en: {ruta_guardado}")

    return imagen, filename


IndentationError: unexpected indent (ipython-input-3711120671.py, line 8)

In [ ]:
# ============================================================================
#  FUNCIÓN: DETECTAR LANDMARKS FACIALES CON MEDIAPIPE
# ============================================================================

def detectar_landmarks(imagen):
    """
    Detecta los landmarks faciales de una imagen usando MediaPipe Face Mesh.

    Parámetros:
        imagen (ndarray): Imagen en formato OpenCV (BGR)

    Retorna:
        dict: Diccionario con coordenadas en píxeles de cada landmark definido
              en LANDMARK_MAPPING, por ejemplo:
              {'nasion': (x, y), 'pronasale': (x, y), ...}
              Si no se detecta ningún rostro → None
    """
    print("\n PASO 2: Detectando landmarks faciales...")

    # --- 1️ Convertir BGR a RGB (requisito para MediaPipe) ---
    imagen_rgb = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)
    alto, ancho = imagen_rgb.shape[:2]

    # --- 2️ Procesar la imagen con MediaPipe Face Mesh ---
    resultados = face_mesh.process(imagen_rgb)

    if not resultados.multi_face_landmarks:
        print(" No se detectó ningún rostro en la imagen.")
        return None

    if len(resultados.multi_face_landmarks) > 1:
        print(" Se detectaron múltiples rostros. Se tomará el primero.")

    # --- 3️ Extraer landmarks del primer rostro detectado ---
    face_landmarks = resultados.multi_face_landmarks[0]
    landmarks_coords = {}

    # --- 4️ Convertir landmarks normalizados a coordenadas en píxeles ---
    for nombre_punto, indice_mp in LANDMARK_MAPPING.items():
        if indice_mp < len(face_landmarks.landmark):
            landmark = face_landmarks.landmark[indice_mp]
            x_px = int(landmark.x * ancho)
            y_px = int(landmark.y * alto)
            landmarks_coords[nombre_punto] = (x_px, y_px)

    print(f" Landmarks detectados: {len(landmarks_coords)} puntos")

    # --- 5️ Mostrar algunos puntos clave (opcional) ---
    puntos_clave = ['nasion', 'pronasale', 'ectocanthion_L', 'ectocanthion_R']
    for punto in puntos_clave:
        if punto in landmarks_coords:
            x, y = landmarks_coords[punto]
            print(f"   📍 {punto}: ({x}, {y})")

    return landmarks_coords


In [ ]:
# ============================================================================
# FUNCIÓN: CALCULAR MEDIDAS MORFOMÉTRICAS EN PÍXELES
# ============================================================================

import math

def calcular_medidas(landmarks):
    """
    Calcula distancias morfométricas (en píxeles) entre puntos anatómicos
    definidos en MEDIDAS_OSTEOFORENSES.

    Parámetros:
        landmarks (dict): Diccionario con coordenadas de landmarks,
                          ej: {'nasion': (x,y), 'pronasale': (x,y)}

    Retorna:
        dict: Un diccionario con las medidas calculadas, cada entrada incluye:
              - descripción
              - nombre de los puntos A y B
              - coordenadas de A y B
              - distancia en píxeles
    """
    print("\n PASO 3: Calculando medidas morfométricas...")

    # --- 1️Función interna para calcular distancia euclidiana ---
    def distancia_euclidiana(punto1, punto2):
        return math.sqrt(
            (punto1[0] - punto2[0])**2 +
            (punto1[1] - punto2[1])**2
        )

    medidas_calculadas = {}
    medidas_exitosas = 0

    # --- 2️ Calcular cada medida definida en MEDIDAS_OSTEOFORENSES ---
    for codigo_medida, (punto_a, punto_b, descripcion) in MEDIDAS_OSTEOFORENSES.items():
        if punto_a in landmarks and punto_b in landmarks:
            try:
                distancia_px = distancia_euclidiana(
                    landmarks[punto_a], landmarks[punto_b])

                # Guardar resultados en el diccionario
                medidas_calculadas[codigo_medida] = {
                    'descripcion': descripcion,
                    'punto_a': punto_a,
                    'punto_b': punto_b,
                    'coord_a': landmarks[punto_a],
                    'coord_b': landmarks[punto_b],
                    'distancia_px': round(distancia_px, 3)
                }
                medidas_exitosas += 1

            except Exception as e:
                print(f" Error calculando {codigo_medida}: {e}")
        else:
            # Si faltan puntos para la medida, mostrar alerta
            puntos_faltantes = []
            if punto_a not in landmarks:
                puntos_faltantes.append(punto_a)
            if punto_b not in landmarks:
                puntos_faltantes.append(punto_b)
            print(f" {codigo_medida}: Faltan puntos {puntos_faltantes}")

    print(f" Medidas calculadas con éxito: {medidas_exitosas}/{len(MEDIDAS_OSTEOFORENSES)}")

    # --- 3️ Mostrar medidas principales (opcional) ---
    medidas_principales = ['ex_ex', 'n_sn', 'al_al', 'ch_ch']
    print("📏 Medidas principales (en píxeles):")
    for medida in medidas_principales:
        if medida in medidas_calculadas:
            valor = medidas_calculadas[medida]['distancia_px']
            desc = medidas_calculadas[medida]['descripcion']
            print(f"   • {medida}: {valor} px → {desc}")

    return medidas_calculadas


In [ ]:
# ============================================================================
# FUNCIÓN: CONVERTIR MEDIDAS DE PÍXELES A MILÍMETROS
# ============================================================================

def convertir_a_milimetros(medidas_px, referencia_mm=96):
    """
    Convierte las medidas calculadas en píxeles a milímetros usando
    una referencia conocida (por defecto, la distancia bi-ectocanthal = 96 mm).

    Parámetros:
        medidas_px (dict): Diccionario de medidas en píxeles generado por calcular_medidas().
        referencia_mm (float): Valor en milímetros de la distancia bi-ectocanthal usada para calibrar.

    Retorna:
        tuple:
          - medidas_mm (dict): Medidas con distancias convertidas a mm.
          - factor_escala (float): Factor de conversión mm/px.
    """
    print("\n PASO 4: Convirtiendo medidas de píxeles a milímetros...")

    # --- 1️ Verificar medida de calibración (bi-ectocanthal) ---
    if 'ex_ex' not in medidas_px:
        print(" No se encontró la medida bi-ectocanthal (ex_ex) para calibración.")
        return None, None

    # --- 2️ Calcular factor de escala mm/px ---
    ex_ex_px = medidas_px['ex_ex']['distancia_px']
    factor_escala = referencia_mm / ex_ex_px  # milímetros por píxel

    print("📐 Parámetros de calibración:")
    print(f"   📏 Bi-ectocanthal medido: {ex_ex_px} píxeles")
    print(f"   📏 Bi-ectocanthal referencia: {referencia_mm} mm")
    print(f"   🔢 Factor de escala: {factor_escala:.6f} mm/px")

    # --- 3️ Convertir todas las medidas a milímetros ---
    medidas_mm = {}
    for codigo_medida, datos in medidas_px.items():
        distancia_mm = datos['distancia_px'] * factor_escala

        # Copiamos los datos originales y añadimos campos de milímetros
        medidas_mm[codigo_medida] = {
            **datos,
            'distancia_mm': round(distancia_mm, 2),
            'factor_escala_mm_px': round(factor_escala, 6),
            'referencia_calibracion': f"bi-ectocanthal = {referencia_mm} mm"
        }

    print(f" Conversión completada: {len(medidas_mm)} medidas convertidas.")

    # --- 4️ Mostrar medidas clave en milímetros ---
    medidas_clave = ['ex_ex', 'n_sn', 'al_al', 'ch_ch', 'zy_zy', 'g_gn']
    print("📏 Medidas principales (milímetros):")
    for medida in medidas_clave:
        if medida in medidas_mm:
            valor_mm = medidas_mm[medida]['distancia_mm']
            desc = medidas_mm[medida]['descripcion']
            print(f"   • {medida}: {valor_mm} mm → {desc}")

    return medidas_mm, factor_escala


In [ ]:
# ============================================================================
#  FUNCIÓN: CREAR VISUALIZACIÓN DE LANDMARKS Y MEDICIONES
# ============================================================================
def crear_visualizacion(imagen, landmarks, medidas):
    """
    Genera una visualización comparativa entre:
      - Imagen original
      - Imagen con puntos (landmarks) y líneas de medición superpuestos.

    Parámetros:
        imagen (np.ndarray): Imagen original en formato BGR.
        landmarks (dict): Diccionario {nombre_punto: (x, y)} con coordenadas.
        medidas (dict): Diccionario con datos de medidas (px y mm).

    Retorna:
        img_viz (np.ndarray): Imagen con landmarks y mediciones dibujadas.
    """
    print("\n PASO 5: Creando visualización...")

    # --- 1️ Crear copia de la imagen para no modificar la original ---
    img_viz = imagen.copy()

    # --- 2️ Dibujar LANDMARKS ---
    for nombre, (x, y) in landmarks.items():
        # Punto verde (centro)
        cv2.circle(img_viz, (x, y), 4, (0, 255, 0), -1)
        # Borde negro para resaltar
        cv2.circle(img_viz, (x, y), 4, (0, 0, 0), 1)
        # Etiqueta con primeras 4 letras del punto
        cv2.putText(img_viz, nombre[:4], (x + 6, y - 6),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.35, (255, 255, 255), 1)
        cv2.putText(img_viz, nombre[:4], (x + 5, y - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 0), 1)

    # --- 3️ Definir colores para MEDIDAS PRINCIPALES ---
    medidas_principales = {
        'ex_ex': (255, 0, 0),      # Rojo - referencia
        'n_sn': (0, 255, 0),       # Verde - altura nasal
        'al_al': (0, 0, 255),      # Azul - ancho nasal
        'ch_ch': (255, 255, 0),    # Amarillo - ancho boca
        'zy_zy': (255, 0, 255),    # Magenta - bizigomático
        'g_gn': (0, 255, 255)      # Cian - altura facial
    }

    # --- 4️ Dibujar líneas y textos de MEDICIONES ---
    for medida, color in medidas_principales.items():
        if medida in medidas:
            datos = medidas[medida]
            pt1 = datos['coord_a']
            pt2 = datos['coord_b']

            # Línea entre puntos
            cv2.line(img_viz, pt1, pt2, color, 2)

            # Calcular punto medio para colocar texto
            punto_medio = ((pt1[0] + pt2[0]) // 2,
                           (pt1[1] + pt2[1]) // 2)
            texto_medida = f"{datos['distancia_mm']} mm"

            # Fondo blanco para el texto
            (w_texto, h_texto), _ = cv2.getTextSize(texto_medida,
                                                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1)
            cv2.rectangle(img_viz,
                          (punto_medio[0] - w_texto // 2 - 2, punto_medio[1] - h_texto - 2),
                          (punto_medio[0] + w_texto // 2 + 2, punto_medio[1] + 2),
                          (255, 255, 255), -1)

            # Texto negro encima
            cv2.putText(img_viz, texto_medida,
                        (punto_medio[0] - w_texto // 2, punto_medio[1]),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

    # --- 5️ Mostrar comparativa: Original vs Analizada ---
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

    # Imagen original
    ax1.imshow(cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB))
    ax1.set_title("Imagen Original", fontsize=14, fontweight='bold')
    ax1.axis('off')

    # Imagen con análisis
    ax2.imshow(cv2.cvtColor(img_viz, cv2.COLOR_BGR2RGB))
    ax2.set_title("Análisis Osteoforense - Landmarks y Mediciones",
                  fontsize=14, fontweight='bold')
    ax2.axis('off')

    fig.suptitle("Análisis Osteoforense - Hash: 0aad2507...68b5",
                 fontsize=16, fontweight='bold')

    plt.tight_layout()
    plt.show()

    # --- 6️ Guardar la visualización ---
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    ruta_viz = f"{proyecto_path}/resultados/visualizacion_{timestamp}.jpg"
    cv2.imwrite(ruta_viz, img_viz)
    print(f" Visualización guardada en: {ruta_viz}")

    return img_viz


In [ ]:
# ============================================================================
#  FUNCIÓN: EXPORTAR BASE DE DATOS DE ANÁLISIS OSTEFORENSE
# ============================================================================
def exportar_base_datos(landmarks, medidas_mm, factor_escala, filename):
    """
    Exporta los resultados del análisis osteoforense en varios formatos (Excel, CSV, JSON).

    Parámetros:
        landmarks (dict): Puntos anatómicos detectados {nombre: (x, y)}.
        medidas_mm (dict): Medidas morfométricas ya convertidas a milímetros.
        factor_escala (float): Factor mm/px usado en la conversión.
        filename (str): Nombre del archivo original analizado.

    Retorna:
        pd.DataFrame: DataFrame con todas las medidas calculadas.
    """
    print("\n PASO 6: Exportando base de datos...")

    # === 1️ Preparar metadatos ===
    timestamp = datetime.now()
    hash_imagen = "0aad2507592c642223603f2e46cf1bdc601f8136fd5e7a4cc4cec59f844a68b5"

    # === 2️ Convertir medidas a filas de DataFrame ===
    filas_datos = []
    for codigo_medida, datos in medidas_mm.items():
        fila = {
            'timestamp': timestamp,
            'hash_imagen': hash_imagen,
            'archivo_original': filename,
            'codigo_medida': codigo_medida,
            'descripcion': datos['descripcion'],
            'punto_a': datos['punto_a'],
            'punto_b': datos['punto_b'],
            'coord_a_x': datos['coord_a'][0],
            'coord_a_y': datos['coord_a'][1],
            'coord_b_x': datos['coord_b'][0],
            'coord_b_y': datos['coord_b'][1],
            'distancia_px': datos['distancia_px'],
            'distancia_mm': datos['distancia_mm'],
            'factor_escala_mm_px': datos['factor_escala_mm_px'],
            'referencia_calibracion': datos['referencia_calibracion']
        }
        filas_datos.append(fila)

    # Crear DataFrame principal con medidas
    df_resultados = pd.DataFrame(filas_datos)

    # === 3️ Preparar nombres de archivos ===
    base_nombre = f"analisis_osteoforense_{timestamp.strftime('%Y%m%d_%H%M%S')}"
    ruta_excel = f"{proyecto_path}/resultados/{base_nombre}.xlsx"
    ruta_csv = f"{proyecto_path}/resultados/{base_nombre}.csv"
    ruta_json = f"{proyecto_path}/resultados/{base_nombre}.json"

    # === 4️ Exportar Excel con 3 hojas ===
    with pd.ExcelWriter(ruta_excel, engine='openpyxl') as writer:
        # Hoja principal con medidas
        df_resultados.to_excel(writer, sheet_name='Medidas_Completas', index=False)

        # Hoja de landmarks
        df_landmarks = pd.DataFrame([
            {'landmark': nombre, 'x_px': coord[0], 'y_px': coord[1]}
            for nombre, coord in landmarks.items()
        ])
        df_landmarks.to_excel(writer, sheet_name='Landmarks', index=False)

        # Hoja resumen solo con medida y distancia mm
        df_resumen = df_resultados[['codigo_medida', 'descripcion', 'distancia_mm']].copy()
        df_resumen.to_excel(writer, sheet_name='Resumen_Medidas', index=False)

    # === 5️ Exportar CSV ===
    df_resultados.to_csv(ruta_csv, index=False, encoding='utf-8')

    # === 6️ Exportar JSON ===
    datos_json = {
        'metadatos': {
            'timestamp': timestamp.isoformat(),
            'hash_imagen': hash_imagen,
            'archivo_original': filename,
            'total_landmarks': len(landmarks),
            'total_medidas': len(medidas_mm),
            'factor_escala_mm_px': factor_escala,
            'referencia_calibracion': 'bi-ectocanthal = 96mm'
        },
        'landmarks': {nombre: {'x_px': coord[0], 'y_px': coord[1]}
                      for nombre, coord in landmarks.items()},
        'medidas_morfometricas': medidas_mm
    }

    with open(ruta_json, 'w', encoding='utf-8') as f:
        json.dump(datos_json, f, indent=2, ensure_ascii=False, default=str)

    # === 7️ Mostrar reporte en consola ===
    print("\n" + "=" * 60)
    print(" ANÁLISIS COMPLETADO - REPORTE FINAL")
    print("=" * 60)
    print(f" Imagen analizada: {filename}")
    print(f" Hash: {hash_imagen}")
    print(f" Timestamp: {timestamp.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"📍 Landmarks detectados: {len(landmarks)}")
    print(f"📏 Medidas calculadas: {len(medidas_mm)}")
    print(f"🔢 Factor de escala: {factor_escala:.6f} mm/px")

    print(f"\n Archivos exportados:")
    print(f"    Excel: {base_nombre}.xlsx")
    print(f"    CSV: {base_nombre}.csv")
    print(f"    JSON: {base_nombre}.json")

    # Mostrar algunas medidas clave en consola
    print(f"\n📏 MEDIDAS MORFOMÉTRICAS FINALES:")
    medidas_reporte = ['g_gn', 'zy_zy', 'go_go', 'n_sn', 'al_al', 'ex_ex', 'ch_ch']
    for medida in medidas_reporte:
        if medida in medidas_mm:
            valor = medidas_mm[medida]['distancia_mm']
            descripcion = medidas_mm[medida]['descripcion']
            print(f"    {medida}: {valor} mm - {descripcion}")

    print("=" * 60)

    return df_resultados


In [ ]:
# ============================================================================
# FUNCIÓN PRINCIPAL: EJECUTAR ANÁLISIS COMPLETO
# ============================================================================
def ejecutar_analisis_completo():
    """
    Ejecuta todo el flujo del análisis osteoforense de una imagen:
    1. Carga de imagen
    2. Detección de landmarks
    3. Cálculo de medidas en píxeles
    4. Conversión de medidas a milímetros
    5. Visualización gráfica del análisis
    6. Exportación de resultados a base de datos (Excel, CSV, JSON)

    Retorna:
        bool: True si todo se ejecutó correctamente, False si hubo algún error.
    """

    print("\n INICIANDO ANÁLISIS OSTEOFORENSE - IMAGEN INDIVIDUAL")
    print(" Hash de referencia: 0aad2507592c642223603f2e46cf1bdc601f8136fd5e7a4cc4cec59f844a68b5")
    print("=" * 80)

    try:
        # === 1️ Paso 1: Cargar imagen ===
        imagen, filename = cargar_imagen()
        if imagen is None:
            print(" No se pudo cargar la imagen. Proceso detenido.")
            return False

        # === 2️ Paso 2: Detectar landmarks ===
        landmarks = detectar_landmarks(imagen)
        if landmarks is None:
            print(" No se pudieron detectar landmarks. Proceso detenido.")
            return False

        # === 3️ Paso 3: Calcular medidas en píxeles ===
        medidas_px = calcular_medidas(landmarks)
        if not medidas_px:
            print(" No se pudieron calcular medidas. Proceso detenido.")
            return False

        # === 4️ Paso 4: Convertir medidas a milímetros ===
        medidas_mm, factor_escala = convertir_a_milimetros(medidas_px)
        if medidas_mm is None:
            print(" No se pudo realizar calibración. Proceso detenido.")
            return False

        # === 5️ Paso 5: Crear visualización del análisis ===
        crear_visualizacion(imagen, landmarks, medidas_mm)

        # === 6️ Paso 6: Exportar resultados a base de datos ===
        df_resultados = exportar_base_datos(landmarks, medidas_mm, factor_escala, filename)

        # ===  Finalización ===
        print("\n ¡ANÁLISIS COMPLETADO EXITOSAMENTE!")
        print(" Resultados almacenados y visualización generada correctamente.")
        return True

    except Exception as e:
        # Manejo de errores críticos
        print("\n ERROR CRÍTICO DURANTE EL ANÁLISIS")
        print(f" Detalles del error: {e}")
        return False


In [ ]:
# ============================================================================
#  EJECUTAR ANÁLISIS COMPLETO DESDE TERMINAL O NOTEBOOK
# ============================================================================
if __name__ == "__main__":
    print("\n Iniciando ejecución completa del análisis osteoforense...")
    print("=" * 80)

    exito = ejecutar_analisis_completo()  # Llama a la función principal

    if exito:
        print("\n PROCESO FINALIZADO CORRECTAMENTE")
        print(" Todos los resultados, gráficos y bases de datos se han guardado en tu Google Drive")
        print(" Revisa la carpeta 'Analisis_Osteoforense/resultados' para ver los archivos generados.")
    else:
        print("\n PROCESO FINALIZADO CON ERRORES")
        print(" Verifica que la imagen cumpla estas condiciones:")
        print("   - Contenga un rostro claramente visible")
        print("   - Buena iluminación y calidad de imagen")
        print("   - Puntos faciales detectables (sin accesorios que los tapen)")
        print("\n Vuelve a ejecutar el análisis una vez verificado todo.")



 Iniciando ejecución completa del análisis osteoforense...

 INICIANDO ANÁLISIS OSTEOFORENSE - IMAGEN INDIVIDUAL
 Hash de referencia: 0aad2507592c642223603f2e46cf1bdc601f8136fd5e7a4cc4cec59f844a68b5
PASO 1: Cargar imagen para análisis
Selecciona la imagen que deseas analizar:


Saving 39f47eace8589a2f2730bbd1a4cd46daf6422e324890263690f38c6f424616f5.png to 39f47eace8589a2f2730bbd1a4cd46daf6422e324890263690f38c6f424616f5.png
Imagen cargada exitosamente.
   📏 Dimensiones: 369 x 369 píxeles
Archivo: 39f47eace8589a2f2730bbd1a4cd46daf6422e324890263690f38c6f424616f5.png
Imagen guardada en: /content/drive/MyDrive/Analisis_Osteoforense/imagenes/0aad2507592c642223603f2e46cf1bdc601f8136fd5e7a4cc4cec59f844a68b5.jpg

 PASO 2: Detectando landmarks faciales...
 Landmarks detectados: 21 puntos
   📍 nasion: (144, 164)
   📍 pronasale: (144, 223)
   📍 ectocanthion_L: (91, 163)
   📍 ectocanthion_R: (216, 157)

 ERROR CRÍTICO DURANTE EL ANÁLISIS
 Detalles del error: name 'calcular_medidas' is not defined

 PROCESO FINALIZADO CON ERRORES
 Verifica que la imagen cumpla estas condiciones:
   - Contenga un rostro claramente visible
   - Buena iluminación y calidad de imagen
   - Puntos faciales detectables (sin accesorios que los tapen)

 Vuelve a ejecutar el análisis una vez verific